# Create your class that implements the Gradient Boosting concept, based on the locally weighted regression method (Lowess class), and that allows a user-prescribed number of boosting steps. The class you develop should have all the mainstream useful options, including “fit,” “is_fitted”, and “predict,” methods. Show applications with real data for regression, 10-fold cross-validations and compare the effect of different scalers, such as the “StandardScaler”, “MinMaxScaler”, and the “QuantileScaler”. In the case of the “Concrete” data set, determine a choice of hyperparameters that yield lower MSEs for your method when compared to the eXtream Gradient Boosting library.

In [1]:
# computational libraries
import numpy as np
import pandas as pd
import xgboost
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import warnings
warnings.filterwarnings('ignore')

from sklearn import linear_model
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

In [ ]:
std_scale = StandardScaler()
quant_scale = QuantileTransformer()
minmax_scale = MinMaxScaler()

scalers = [std_scale, quant_scale, minmax_scale]

In [ ]:
# Gaussian Kernel
def Gaussian(w):
  return np.where(w>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*w**2))

# Tricubic Kernel
def Tricubic(w):
  return np.where(w>1,0,70/81*(1-w**3)**3)

# Quartic Kernel
def Quartic(w):
  return np.where(w>1,0,15/16*(1-w**2)**2)

# Epanechnikov Kernel
def Epanechnikov(w):
  return np.where(w>1,0,3/4*(1-w**2))

In [ ]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(cdist(u, v, metric='euclidean')/(2*tau))

In [ ]:
# # here we have a function that computes the Euclidean distance between all the observations in u, and v
# def dist(u,v):
#   return cdist(u, v, metric='euclidean')

# # how about the case when v is a matrix?
# def dist(u,v):
#   D = []
#   if len(v.shape)==1:
#     v = v.reshape(1,-1)
#   # we would like all the pairwise combinations if u and v are matrices
#   # we could avoid two for loops if we consider broadcasting
#   for rowj in v:
#     D.append(np.sqrt(np.sum((u-rowj)**2,axis=1)))
#   return np.array(D).T

# here we have a function that computes the Euclidean distance between all the observations in u, and v
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [ ]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]
  # dist(x,x) is always symmetric
  w = np.clip(dist(x,x) / np.array(h), 0.0, 1.0)
  # note that w is a square matrix and in Python arithmetic operations such as
  # w**3 or 1-w**3 are performed element-wise
  #w = (1-w**3)**3 # a Tricubic kernel
  w = Epanechnikov(w)

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(np.diag(w[i,:]))
      # when we multiply two diagonal matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization for solving the system
      beta = linalg.solve(A, b)

      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta.ravel())

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  # here we are making predictions for xnew by using an interpolation and the predictions we made for the train data
  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ Pp')
      f = LinearNDInterpolator(tri,yest[ind])
      input_i = pca.transform(xnew[i].reshape(1,-1))
      output[i] = f(input_i)
      # the output may have NaN's where the data points from xnew are outside the convex hull of X

  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,yest.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [ ]:
#Lowess Class without triangulation

class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.0001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = []
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test.append(lm.predict([x_new[i]]))
        return np.array(yest_test).flatten()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('drive/MyDrive/concrete.csv')

In [ ]:
data

cement   slag    ash  water  superplastic  coarseagg  fineagg  age  \
0      540.0    0.0    0.0  162.0           2.5     1040.0    676.0   28   
1      540.0    0.0    0.0  162.0           2.5     1055.0    676.0   28   
2      332.5  142.5    0.0  228.0           0.0      932.0    594.0  270   
3      332.5  142.5    0.0  228.0           0.0      932.0    594.0  365   
4      198.6  132.4    0.0  192.0           0.0      978.4    825.5  360   
...      ...    ...    ...    ...           ...        ...      ...  ...   
1025   276.4  116.0   90.3  179.6           8.9      870.1    768.3   28   
1026   322.2    0.0  115.6  196.0          10.4      817.9    813.4   28   
1027   148.5  139.4  108.6  192.7           6.1      892.4    780.0   28   
1028   159.1  186.7    0.0  175.6          11.3      989.6    788.9   28   
1029   260.9  100.5   78.3  200.6           8.6      864.5    761.5   28   

      strength  
0        79.99  
1        61.89  
2        40.27  
3        41.05  
4        44.30  
...        ...  
1025     44.28  
1026     31.18  
1027     23.70  
1028     32.77  
1029     32.40  

[1030 rows x 9 columns]

In [ ]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [ ]:
mse_lwr = []
mse_rf = []
scale = std_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 42.14569000966465
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
mse_lwr = []
mse_rf = []
scale = quant_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 20.84908646431679
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
mse_lwr = []
mse_rf = []
scale = minmax_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 45.368275750156776
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
XGB_MSE=[]
for scale in scalers:
  for idxtrain, idxtest in kf.split(x):
    xtrain = x[idxtrain]
    ytrain = y[idxtrain].ravel()
    ytest = y[idxtest].ravel()
    xtest = x[idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)

  model_xgboost = xgboost.XGBRFRegressor()
  model_xgboost.fit(xtrain,ytrain)
  XGB_MSE.append(mse(ytest,model_xgboost.predict(xtest)))
print(XGB_MSE)

[32.5773262908273, 32.5773262908273, 32.5773262908273]


Since we are only looking at the MSE of the LWR, we see that the middle trial, namely, the one using the scale = QuantileTransformer() along with the Tricubic and Gaussian models achieves a MSE of 20.85 which is lower than the eXtream Gradient Boosting MSE of 32.58.

# Implement your own version of Locally Weighted Logistic Regression and compare its performance on the Iris data set with the version presented in this article: https://calvintchi.github.io/classical_machine_learning/2020/08/16/lwlr.html.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('drive/MyDrive/Iris.csv')

In [4]:
data

Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0      1            5.1           3.5            1.4           0.2   
1      2            4.9           3.0            1.4           0.2   
2      3            4.7           3.2            1.3           0.2   
3      4            4.6           3.1            1.5           0.2   
4      5            5.0           3.6            1.4           0.2   
..   ...            ...           ...            ...           ...   
145  146            6.7           3.0            5.2           2.3   
146  147            6.3           2.5            5.0           1.9   
147  148            6.5           3.0            5.2           2.0   
148  149            6.2           3.4            5.4           2.3   
149  150            5.9           3.0            5.1           1.8   

            Species  
0       Iris-setosa  
1       Iris-setosa  
2       Iris-setosa  
3       Iris-setosa  
4       Iris-setosa  
..              ...  
145  Iris-virginica  
146  Iris-virginica  
147  Iris-virginica  
148  Iris-virginica  
149  Iris-virginica  

[150 rows x 6 columns]

In [5]:
# Libraries of functions need to be imported
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy import linalg
from scipy.interpolate import interp1d, LinearNDInterpolator, NearestNDInterpolator
from sklearn.decomposition import PCA

# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [6]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

# this is the correct vectorized version
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))

# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

In [7]:
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
    d = np.sqrt(np.sum((u-v)**2,axis=1))
  else:
    d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [8]:
def kernel_function(xi,x0,kern, tau):
    return kern(dist(xi,x0)/(2*tau))

In [9]:
def weights_matrix(x,x_new,kern,tau):
  if np.isscalar(x_new):
    w = kernel_function(x,x_new,kern,tau)
  else:
    if len(x_new.shape)==1:
      n = 1
      w = kernel_function(x,x_new,kern,tau=0.05).reshape(1,-1)
    else:
      n = len(x_new)
      w = np.array([kernel_function(x,x_new[i],kern,tau) for i in range(n)]).reshape(n,len(x))
  return w

In [10]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(dist(u,v)/(2*tau))

In [11]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test[i] = lm.predict(x_new[i].reshape(1,-1))
        return yest_test

In [13]:
from sklearn.preprocessing import LabelEncoder

x = data.loc[:,'SepalLengthCm':'PetalWidthCm'].values
y = data['Species'].values

y = LabelEncoder().fit_transform(y)

scale = StandardScaler()
model = Lowess(kernel=Epanechnikov,tau=0.02)
xscaled = scale.fit_transform(x)
model.fit(xscaled,y)
yhat = model.predict(xscaled)
mse(yhat,y)

5.493260742965061e-06

The Mean Square Error is 5.493260742965061e-06 which is a very small number so we are very pleased with this result.

Below we will check the Mean Square Error of Calvin Chi's model found here: https://calvintchi.github.io/classical_machine_learning/2020/08/16/lwlr.html

In [15]:
import numpy as np
import pandas as pd
import pylab as pl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# DATA_DIR = "datasets/CS229_P1"
# X = pd.DataFrame(np.loadtxt(DATA_DIR + "/x.dat"))
# Y = pd.DataFrame(np.loadtxt(DATA_DIR + "/y.dat"))

In [16]:
class locally_weighted_logistic_regression(object):

    def __init__(self, tau, reg = 0.0001, threshold = 1e-6):
        self.reg = reg
        self.threshold = threshold
        self.tau = tau
        self.w = None
        self.theta = None
        self.x = None

    def weights(self, x_train, x):
        sq_diff = (x_train - x)**2
        norm_sq = sq_diff.sum(axis = 1)
        return np.ravel(np.exp(- norm_sq / (2 * self.tau**2)))

    def logistic(self, x_train):
        return np.ravel(1 / (1 + np.exp(-x_train.dot(self.theta))))

    def train(self, x_train, y_train, x):
        self.w = self.weights(x_train, x)
        self.theta = np.zeros(x_train.shape[1])
        self.x = x
        gradient = np.ones(x_train.shape[1]) * np.inf
        while np.linalg.norm(gradient) > self.threshold:
            # compute gradient
            h = self.logistic(x_train)
            gradient = x_train.T.dot(self.w * (np.ravel(y_train) - h)) - self.reg * self.theta
            # Compute Hessian
            D = np.diag(-(self.w * h * (1 - h)))
            H = x_train.T.dot(D).dot(x_train) - self.reg * np.identity(x_train.shape[1])
            # weight update
            self.theta = self.theta - np.linalg.inv(H).dot(gradient)

    def predict(self):
        return np.array(self.logistic(self.x) > 0.5).astype(int)

In [21]:
scale = StandardScaler()
model = locally_weighted_logistic_regression(tau=0.02)
xscaled = scale.fit_transform(x)
model.train(xscaled, y, xscaled)
yhat = model.predict()
mse = mean_squared_error(y, yhat)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.38


So our model actually gives a smaller Mean Square Error than Calvin's!